In [1]:
import pandas as pd
import os, urllib,re
from pymol import cmd,stored
from rdkit import Chem
import json

In [ ]:
#table=pd.read_excel('Cys_DataBase.xlsx')

In [2]:
for file in os.listdir('DATA/Scorpion/'):
    if '.pml' in file:
        entry=file.split('.')[0]
        with open('DATA/Scorpion/'+file, 'r') as f:
            doc=[line for line in f.readlines()]

            data={'Residues':{},
                  'Interactions': {},
                  'Structures': {},
                  'Water_ranks':{}}
            keys=[]
            hbond=[]
            hdon_pi=[]
            ionic=[]
            pi_pi=[]
            poor_ang=[]
            unclass=[]
            unfav=[]
            vdW=[]
            wat=[]
            my_dict = { 'Residues' : [] }

            for line in doc:

                if "print 'Scorpion Total:" in line:
                    scorpion_score=float (line.split(':')[-1].replace("'",''))
                    data['Scorpion_score']=scorpion_score

                if 'bs_protein.pdb' in line:
                    urls = re.findall(r'(https?://[^\s]+)',line)
                    url=urls[0].split('"')[0]

                if 'label /water_rank/' in line:
                    l=line.replace("label /water_rank//",'').replace(", '         %4.1f' %",'').split(' ')
                    wat.append((l[0].replace('/','_').replace('`',''),float(l[1])))

                line=line.replace('\n','').split(',')
                if len(line) ==3:
                    if line[0].split(' ')[1] == 'hbond':
                        hbond.append("_".join(line[1].replace('`','').split('/')[3:])+'/'+"_".join(line[2].replace('`','').split('/')[3:]))
                    if line[0].split(' ')[1] == 'hdon_pi':
                        hdon_pi.append("_".join(line[1].replace('`','').split('/')[3:])+'/'+"_".join(line[2].replace('`','').split('/')[3:]))
                    if line[0].split(' ')[1] == 'ionic':
                        ionic.append("_".join(line[1].replace('`','').split('/')[3:])+'/'+"_".join(line[2].replace('`','').split('/')[3:]))
                    if line[0].split(' ')[1] == 'pi_pi':
                        pi_pi.append("_".join(line[1].replace('`','').split('/')[3:])+'/'+"_".join(line[2].replace('`','').split('/')[3:]))
                    if line[0].split(' ')[1] == 'poor_ang':
                        poor_ang.append("_".join(line[1].replace('`','').split('/')[3:])+'/'+"_".join(line[2].replace('`','').split('/')[3:]))
                    if line[0].split(' ')[1] == 'unclass':
                        unclass.append("_".join(line[1].replace('`','').split('/')[3:])+'/'+"_".join(line[2].replace('`','').split('/')[3:]))
                    if line[0].split(' ')[1] == 'unfav':
                        unfav.append("_".join(line[1].replace('`','').split('/')[3:])+'/'+"_".join(line[2].replace('`','').split('/')[3:]))
                    if line[0].split(' ')[1] == 'vdW':
                        vdW.append("_".join(line[1].replace('`','').split('/')[3:])+'/'+"_".join(line[2].replace('`','').split('/')[3:]))

            data['Interactions']['hbond']=hbond
            data['Interactions']['hdon_pi']=hdon_pi
            data['Interactions']['ionic']=hbond
            data['Interactions']['pi_pi']=hbond
            data['Interactions']['vdW']=vdW
            data['Interactions']['poor_ang']=poor_ang
            data['Interactions']['unclass']=unclass
            data['Interactions']['unfav']=unfav
            data['Water_ranks']=wat
            filename = urllib.urlretrieve(url)[0]
            cmd.load(filename, "struct")
            cmd.create ('ref', 'struct')

            cmd.iterate("backbone","Residues.append((resn,resi))",space=my_dict)
            f=[", ".join(i).replace(', ','') for i in my_dict['Residues']]
            data['Residues']=list(set(f))

            cmd.create ('ligand', 'struct and hetatm')
            cmd.remove ('struct in ligand')
            ligand_name='DATA/Ligands/'+entry.upper()+'.sdf'
            cmd.save (ligand_name,'ligand',state=0,format='sdf')
            cmd.delete('all')
            data['Structures']['sdf']=[line for line in open(ligand_name,'r').readlines()]
            data['Structures']['smiles']=Chem.MolToSmiles(Chem.SDMolSupplier(ligand_name,False)[0])
        with open('DATA/Interactions/'+entry.upper()+'.json', 'w') as fp:
            json.dump(data, fp, sort_keys=True, indent=4)

 PyMOL not running, entering library mode (experimental)
